## Semantic Analysis

NER

In [ ]:
#entity recognition
import spacy

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Example text
text = "Apple Inc. is looking to buy a U.K. startup for $1 billion. Tim Cook announced this yesterday in San Francisco."

# Process the text
doc = nlp(text)

# Display the named entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Apple Inc. ORG
U.K. GPE
$1 billion MONEY
Tim Cook PERSON
this yesterday DATE
San Francisco GPE


Coreference resolution

In [ ]:
!pip install coreferee

In [ ]:
!python -m coreferee install en

2024-04-14 19:39:35.244325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:39:35.244393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:39:35.245853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee-model-en: filename=coreferee_model_en-1.0.0-py3-none-any.whl size=65422507 sha256=fc8eede031dbf318153f8d5f88ea47cfe1f241

In [ ]:
!python -m spacy download en_core_web_md


2024-04-14 19:41:50.736069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:41:50.736151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:41:50.738141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import spacy
import coreferee

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('coreferee')
doc = nlp("Apple is looking at buying a startup. They are interested in its AI technology.")

print(doc._.coref_chains)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  """Check if string maps to a package installed via pip.


[0: [0], [8], 1: [6], [12]]


## Early attempts of fine tunning

In [ ]:
%%capture
!pip install datasets torch

In [ ]:
from datasets import load_dataset

dataset = load_dataset("xsum")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
%%capture
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
model_to_be_fine_tuned = T5ForConditionalGeneration.from_pretrained("t5-small")

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model_to_be_fine_tuned,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(1000)),
    eval_dataset=tokenized_datasets["validation"].select(range(1000)),
    tokenizer=tokenizer_before
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,3.555805
2,No log,0.859342
3,No log,0.827318


TrainOutput(global_step=189, training_loss=4.331529405381945, metrics={'train_runtime': 171.8493, 'train_samples_per_second': 17.457, 'train_steps_per_second': 1.1, 'total_flos': 406025404416000.0, 'train_loss': 4.331529405381945, 'epoch': 3.0})

In [ ]:
fine_tuned_model = model_to_be_fine_tuned

In [ ]:
# Evaluate the fine-tuned model 1000 examples
fine_tuned_scores = evaluate_model(fine_tuned_model, tokenizer_before, dataset)
print("Fine-tuned Model ROUGE Scores:", fine_tuned_scores)

Your max_length is set to 150, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 150, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 150, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
You

Fine-tuned Model ROUGE Scores: {'rouge-1': {'r': 0.22063405265145067, 'p': 0.11705052019213884, 'f': 0.15085126782756195}, 'rouge-2': {'r': 0.021389659535311707, 'p': 0.009645475063951777, 'f': 0.012978985934649568}, 'rouge-l': {'r': 0.18195206454936566, 'p': 0.09598073282756586, 'f': 0.12376672617136635}}


In [ ]:
model_to_be_fine_tuned = T5ForConditionalGeneration.from_pretrained("t5-small")

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model_to_be_fine_tuned,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(10000)),
    eval_dataset=tokenized_datasets["validation"].select(range(10000)),
    tokenizer=tokenizer_before
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,1.891800,0.652877
2,0.722000,0.638455


Epoch,Training Loss,Validation Loss
1,1.891800,0.652877
2,0.722000,0.638455
3,0.701400,0.635526


TrainOutput(global_step=1875, training_loss=1.0237699869791668, metrics={'train_runtime': 1684.0837, 'train_samples_per_second': 17.814, 'train_steps_per_second': 1.113, 'total_flos': 4060254044160000.0, 'train_loss': 1.0237699869791668, 'epoch': 3.0})

In [ ]:
fine_tuned_model = model_to_be_fine_tuned

In [ ]:
# Evaluate the fine-tuned model 1000 examples
fine_tuned_scores = evaluate_model(fine_tuned_model, tokenizer_before, dataset)
print("Fine-tuned Model ROUGE Scores:", fine_tuned_scores)

Your max_length is set to 150, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 150, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 150, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
You

Fine-tuned Model ROUGE Scores: {'rouge-1': {'r': 0.22063405265145067, 'p': 0.11705052019213884, 'f': 0.15085126782756195}, 'rouge-2': {'r': 0.021389659535311707, 'p': 0.009645475063951777, 'f': 0.012978985934649568}, 'rouge-l': {'r': 0.18195206454936566, 'p': 0.09598073282756586, 'f': 0.12376672617136635}}


## Best fine tunning attempt

In [ ]:
%%capture
!pip install transformers
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install nltk

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
train_data = load_dataset("xsum", split="train[:20%]")
test_data = load_dataset("xsum", split="test[:20%]")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/98232 [00:00<?, ? examples/s]

Map:   0%|          | 0/4565 [00:00<?, ? examples/s]

In [ ]:
#
%%capture
!pip install transformers

In [ ]:
#
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration

In [ ]:
#
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
#mount google drive to save the fine tuned model to
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/MyModel",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.178900,2.262195,0.250100,0.077700,0.196000,0.196000,18.862400
2,2.149500,2.243245,0.254600,0.079300,0.199300,0.199300,18.868800
3,2.128700,2.236404,0.255500,0.079500,0.199400,0.199400,18.874700
4,2.148500,2.232910,0.256600,0.080400,0.200600,0.200700,18.890300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=24560, training_loss=2.1661259269093067, metrics={'train_runtime': 21500.6172, 'train_samples_per_second': 18.275, 'train_steps_per_second': 1.142, 'total_flos': 1.0627518714976666e+17, 'train_loss': 2.1661259269093067, 'epoch': 4.0})

In [ ]:
from transformers import pipeline
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
summarizer = pipeline("summarization", model=model)
summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "No one making under $400,000 per year will pay a penny more in taxes than any other tax payer in the U.S. it's the most aggressive action on the climate crisis in America."}]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

path_to_model = '/content/drive/MyDrive/MyModel'

tokenizer = T5Tokenizer.from_pretrained(path_to_model)
model_fine_tuned = T5ForConditionalGeneration.from_pretrained(path_to_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Evaluating models

In [ ]:
#
%%capture
!pip install rouge

In [ ]:
#
from transformers import pipeline
from rouge import Rouge
import torch

def evaluate_model(model, tokenizer, dataset, sample_size=50):

    # If there's a GPU available, move the model to the GPU

    if torch.cuda.is_available():
        print("GPU is available.")
        device = torch.device("cuda")
    else:
        print("GPU not available, using CPU.")
        device = torch.device("cpu")


    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=device)

    # Randomly select a sample of the dataset
    sampled_data = dataset["test"].shuffle(seed=42).select(range(sample_size))

    # Generate summaries
    generated_summaries = []
    for entry in sampled_data:
        summarized = summarizer(entry["document"], max_length=150, truncation=True, min_length=40)
        generated_summaries.append(summarized[0]['summary_text'])

    # Prepare references and candidates for ROUGE
    references = [entry["summary"] for entry in sampled_data]
    candidates = generated_summaries

    rouge = Rouge()
    scores = rouge.get_scores(candidates, references, avg=True)

    # Round each value in the scores to 4 decimal places
    rounded_scores = {key: {metric: round(value, 4) for metric, value in values.items()}
                      for key, values in scores.items()}

    return rounded_scores


In [ ]:
# Evaluate the original t5 model
model_t5.eval()
t5_scores = evaluate_model(model_t5, tokenizer, dataset)

GPU not available, using CPU.


Your max_length is set to 150, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 150, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 150, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
You

In [ ]:
print("Original t5 Model ROUGE Scores:")
for metric, score in t5_scores.items():
    print(f"{metric}: {score}")

Original t5 Model ROUGE Scores:
rouge-1: {'r': 0.2134, 'p': 0.1207, 'f': 0.1513}
rouge-2: {'r': 0.0214, 'p': 0.01, 'f': 0.0133}
rouge-l: {'r': 0.1845, 'p': 0.1037, 'f': 0.1303}


In [ ]:
%%capture
# Evaluate the fine tuned t5 model
model_fine_tuned.eval()
fine_tuned_scores = evaluate_model(model_fine_tuned, tokenizer, dataset)

Your max_length is set to 150, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 150, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 150, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
You

In [ ]:
print("Fine-tuned Model ROUGE Scores:")
for metric, score in fine_tuned_scores.items():
    print(f"{metric}: {score}")

Fine-tuned Model ROUGE Scores:
rouge-1: {'r': 0.3132, 'p': 0.2035, 'f': 0.2408}
rouge-2: {'r': 0.0708, 'p': 0.0398, 'f': 0.0497}
rouge-l: {'r': 0.2577, 'p': 0.166, 'f': 0.1971}


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
tokenizer_bart = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
# Evaluate the BART model
model_bart.eval()
tokenizer_bart.model_max_length = 1024
bart_scores = evaluate_model(model_bart, tokenizer_bart, dataset)

GPU not available, using CPU.


Your max_length is set to 150, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 150, but your input_length is only 70. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)
Your max_length is set to 150, but your input_length is only 145. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)
Your max_length is set to 150, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
You

In [ ]:
print("BART Model ROUGE Scores:")
for metric, score in bart_scores.items():
    print(f"{metric}: {score}")

BART Model ROUGE Scores:
rouge-1: {'r': 0.2496, 'p': 0.1398, 'f': 0.1766}
rouge-2: {'r': 0.0353, 'p': 0.0168, 'f': 0.0223}
rouge-l: {'r': 0.2234, 'p': 0.1245, 'f': 0.1575}


In [ ]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

model_name = 'facebook/blenderbot-400M-distill'
tokenizer_blenderbot = BlenderbotTokenizer.from_pretrained(model_name)
model_blenderbot = BlenderbotForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

In [ ]:
# Evaluate the BART model
model_blenderbot.eval()
blenderbot_scores = evaluate_model(model_blenderbot, tokenizer_blenderbot, dataset)

Your max_length is set to 150, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


GPU not available, using CPU.


Your max_length is set to 150, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)
Your max_length is set to 150, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)
Your max_length is set to 150, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)
Your max_length is set to 150, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


In [ ]:
print("BlenderBot Model ROUGE Scores:")
for metric, score in blenderbot_scores.items():
    print(f"{metric}: {score}")

BlenderBot Model ROUGE Scores:
rouge-1: {'r': 0.1155, 'p': 0.0929, 'f': 0.1016}
rouge-2: {'r': 0.0042, 'p': 0.0034, 'f': 0.0037}
rouge-l: {'r': 0.1021, 'p': 0.0819, 'f': 0.0896}


# For demo

In [ ]:
from transformers import pipeline
from rouge import Rouge
import torch

def summarize(model,tokenizer, text):
  if torch.cuda.is_available():
        print("GPU is available.")
        device = torch.device("cuda")
  else:
      print("GPU not available, using CPU.")
      device = torch.device("cpu")

  summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=device)

  summary = summarizer(text, max_length=150, truncation=True, min_length=40)

  return summary

Original t5

In [ ]:
#
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration

In [ ]:
#
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
text = '''Hello my name is kartik. I am studying in IIITB. My name is kartik. Currently staying in banglore. pursuing my MTech specialization.'''

In [ ]:
summarize(model_t5, tokenizer, text)

Your max_length is set to 150, but your input_length is only 41. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


GPU not available, using CPU.


[{'summary_text': 'my name is kartik. I am studying in IIITB. Currently staying in banglore. pursuing my MTech specializationization . if you have any questions, please contact me .'}]

fine tuned t5

In [ ]:
#mount google drive to save the fine tuned model to
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

path_to_model = '/content/drive/MyDrive/MyModel'

tokenizer = T5Tokenizer.from_pretrained(path_to_model)
model_fine_tuned = T5ForConditionalGeneration.from_pretrained(path_to_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
text2 =  "Summarize the following: \""+text+"\""
summarize(model_fine_tuned, tokenizer, text2)

Your max_length is set to 150, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


GPU not available, using CPU.


[{'summary_text': 'My name is kartik. I am studying in IIITB and I am currently staying in banglore. I have a MTech specialization in the field of MTech in the UK.'}]

BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer_bart = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
summarize(model_bart, tokenizer_bart, text)

Your max_length is set to 150, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


GPU not available, using CPU.


[{'summary_text': 'My name is kartik. I am studying in IIITB. Currently staying in banglore. pursuing my MTech specialization. I would like to share some of my experiences with you.'}]

BlenderBot

In [ ]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

model_name = 'facebook/blenderbot-400M-distill'
tokenizer_blenderbot = BlenderbotTokenizer.from_pretrained(model_name)
model_blenderbot = BlenderbotForConditionalGeneration.from_pretrained(model_name)

In [ ]:
text2 =  "Summarize the following: \""+text+"\""
summarize(model_blenderbot, tokenizer_blenderbot, text2)

Your max_length is set to 150, but your input_length is only 51. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


GPU not available, using CPU.


[{'summary_text': " Nice to meet you Kartik! I'm studying to be an engineer. What do you want to do with MT. Specialization? I'm interested in robotics."}]

In [ ]:
text2

'Summarize the following: "Hello my name is kartik. I am studying in IIITB. My name is kartik. Currently staying in banglore. pursuing my MTech specialization."'

In [ ]:
PageRank

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def sent_tokenize_example(example):
    example = sent_tokenize(example)
    return example

In [ ]:
sent_token_text = sent_tokenize_example(text)

In [ ]:
sent_token_text

['Hello my name is kartik.',
 'I am studying in IIITB.',
 'My name is kartik.',
 'Currently staying in banglore.',
 'pursuing my MTech specialization.']

In [ ]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

def summarize_by_page_rank(sentences):

  # Vectorize the sentences using TF-IDF
  vectorizer = TfidfVectorizer()
  sentence_vectors = vectorizer.fit_transform(sentences).toarray()

  # Compute similarity matrix
  similarity_matrix = cosine_similarity(sentence_vectors)

  # Build the graph
  nx_graph = nx.from_numpy_array(similarity_matrix)
  # Apply PageRank
  scores = nx.pagerank(nx_graph)

  # Rank sentences based on scores
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

  # Extract the top N sentences as the summary
  N = 1  # Number of sentences in the summary
  summary = ' '.join([ranked_sentences[i][1] for i in range(N)])
  return summary

In [ ]:
summarize_by_page_rank(sent_token_text)

'My name is kartik.'